# KRX 정보데이터시스템에서 2011~2018 ESG 데이터 가져오기
- [KRX 정보데이터시스템](https://data.krx.co.kr/contents/MDC/HARD/hardController/MDCHARD050.cmd#none)

In [23]:
import requests
import pandas as pd


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
}

df_list = []
for year in range(2011, 2018 + 1):
    data = {'bld': 'dbms/MDC/HARD/MDCHARD05001',
    'locale': 'ko_KR',
    'selTp': '1','grdYy': year}
    response = requests.post('https://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', headers = headers, data = data)
    df_list.append(pd.DataFrame(response.json()['block1']))
df = pd.concat(df_list).drop(columns = 'RN').reset_index(drop = True)
df

,ISU_NM,GRD1,GRD2,GRD3,GRD4,YY
0,AK홀딩스,B이하,B,B이하,B이하,2011
1,BYC,B이하,B,B이하,B이하,2011
2,CJ,B이하,B+,B이하,,2011
3,CJ CGV,B이하,B,B이하,,2011
4,CJ대한통운,B+,B,B+,B이하,2011
...,...,...,...,...,...,...
5649,샘표,B이하,B,B이하,B이하,2018
5650,샘표식품,B+,B,B+,B+,2018
5651,서연,B이하,B,B이하,B이하,2018
5652,서연이화,B+,B+,B+,B이하,2018


In [24]:
df.columns = ['기업명','ESG등급','환경','사회','지배구조','평가년도']

In [25]:
df

,기업명,ESG등급,환경,사회,지배구조,평가년도
0,AK홀딩스,B이하,B,B이하,B이하,2011
1,BYC,B이하,B,B이하,B이하,2011
2,CJ,B이하,B+,B이하,,2011
3,CJ CGV,B이하,B,B이하,,2011
4,CJ대한통운,B+,B,B+,B이하,2011
...,...,...,...,...,...,...
5649,샘표,B이하,B,B이하,B이하,2018
5650,샘표식품,B+,B,B+,B+,2018
5651,서연,B이하,B,B이하,B이하,2018
5652,서연이화,B+,B+,B+,B이하,2018


In [26]:
df.isnull().sum()

기업명      0
ESG등급    0
환경       0
사회       0
지배구조     0
평가년도     0
dtype: int64

In [27]:
df.shape

(5654, 6)

In [28]:
# 중복 데이터 확인
# 평가년도가 다르므로, 놔두기로 함!
df[df.duplicated()]

,기업명,ESG등급,환경,사회,지배구조,평가년도
857,신성이엔지,B이하,C,B이하,B이하,2012
1920,신성이엔지,B이하,C,B이하,B이하,2013
2726,신성이엔지,B이하,C,B이하,B이하,2014


## 평가년도가 2018년이 처음에 오도록 정렬

In [29]:
df = df.sort_values(by="평가년도", ascending=False) 
# https://givemethesocks.tistory.com/25
df = df.reset_index(drop=True)
df

,기업명,ESG등급,환경,사회,지배구조,평가년도
0,서울도시가스,B이하,B,B이하,B이하,2018
1,티비에이치글로벌,B이하,B,B이하,B이하,2018
2,평화산업,B이하,B,B이하,B+,2018
3,페이퍼코리아,B이하,C,B이하,B이하,2018
4,퍼시스,B이하,B,B이하,B이하,2018
...,...,...,...,...,...,...
5649,성신양회,B이하,B+,B이하,B이하,2011
5650,성보화학,,B,,,2011
5651,성문전자,B이하,C,B이하,B이하,2011
5652,선창산업,B이하,B,B이하,B이하,2011


## csv 파일로 저장

In [30]:
df.to_csv("../../data/ESG_Rating(2011-2018).csv", index=False)

In [31]:
esg_1118 = pd.read_csv("../../data/ESG_Rating(2011-2018).csv")
esg_1118

,기업명,ESG등급,환경,사회,지배구조,평가년도
0,서울도시가스,B이하,B,B이하,B이하,2018
1,티비에이치글로벌,B이하,B,B이하,B이하,2018
2,평화산업,B이하,B,B이하,B+,2018
3,페이퍼코리아,B이하,C,B이하,B이하,2018
4,퍼시스,B이하,B,B이하,B이하,2018
...,...,...,...,...,...,...
5649,성신양회,B이하,B+,B이하,B이하,2011
5650,성보화학,NaN,B,NaN,NaN,2011
5651,성문전자,B이하,C,B이하,B이하,2011
5652,선창산업,B이하,B,B이하,B이하,2011


## FinanceDataReader 로 기업코드 가져오기

In [32]:
# FinanceDataReader 를 fdr 별칭으로 불러오기
import FinanceDataReader as fdr

fdr.__version__

'0.9.42'

In [33]:
df_krx = fdr.StockListing("KRX")
df_krx.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,105190,KOSPI,ACE 200,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,332500,KOSPI,ACE 200TR,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,385590,KOSPI,ACE ESG액티브,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,380340,KOSPI,ACE Fn5G플러스,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [34]:
df_krx = df_krx[['Name','Symbol']]
# df_krx = df_krx.loc[df_krx["Market"]=="KOSPI", :]
df_krx.columns = ['Name','기업코드']
df_krx.head()

,Name,기업코드
0,3S,060310
1,ACE 200,105190
2,ACE 200TR,332500
3,ACE ESG액티브,385590
4,ACE Fn5G플러스,380340


In [35]:
# df_krx[df_krx["Name"] == "AJ네트웍스"] # 기업코드 095570

## merge 를 통한 ESG_Rating(2011-2018) 데이터에 기업코드 넣기

In [36]:
df_1118 = esg_1118.merge(df_krx, left_on="기업명", right_on="Name", how="left")
df_1118 = df_1118[["기업명", "기업코드", "ESG등급", "환경", "사회", "지배구조", "평가년도"]]
df_1118.head(2)

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,서울도시가스,NaN,B이하,B,B이하,B이하,2018
1,티비에이치글로벌,NaN,B이하,B,B이하,B이하,2018


In [37]:
df_1118.shape

(5654, 7)

In [38]:
df_1118.tail()

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
5649,성신양회,004980,B이하,B+,B이하,B이하,2011
5650,성보화학,003080,NaN,B,NaN,NaN,2011
5651,성문전자,014910,B이하,C,B이하,B이하,2011
5652,선창산업,NaN,B이하,B,B이하,B이하,2011
5653,AK홀딩스,006840,B이하,B,B이하,B이하,2011


# ESG_Rating(2021).csv 파일 가져오기

In [39]:
df_2021 = pd.read_csv("../../data/ESG_Rating(2021).csv")
df_2021.head(2)

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,95570,B,D,B,B,2021
1,AK홀딩스,6840,B+,B,B+,B+,2021


## 기업코드 6자리로 반환하기

In [40]:
def six_digit(x):
    return '%06d' % x
df_2021['기업코드'] = df_2021['기업코드'].apply(six_digit)

In [41]:
df_2021.head()

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,095570,B,D,B,B,2021
1,AK홀딩스,006840,B+,B,B+,B+,2021
2,AP시스템,265520,D,D,D,C,2021
3,BGF,027410,A,A,A+,A,2021
4,BGF리테일,282330,A,A,A+,A,2021


## concat 을 통한 ESG_Rating 2011-2018 와 2021 데이터 합치기

In [42]:
esg_total = pd.concat([df_2021, df_1118])
esg_total = esg_total.reset_index(drop=True)
esg_total

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,095570,B,D,B,B,2021
1,AK홀딩스,006840,B+,B,B+,B+,2021
2,AP시스템,265520,D,D,D,C,2021
3,BGF,027410,A,A,A+,A,2021
4,BGF리테일,282330,A,A,A+,A,2021
...,...,...,...,...,...,...,...
6654,성신양회,004980,B이하,B+,B이하,B이하,2011
6655,성보화학,003080,NaN,B,NaN,NaN,2011
6656,성문전자,014910,B이하,C,B이하,B이하,2011
6657,선창산업,NaN,B이하,B,B이하,B이하,2011


## 완성된 ESG 데이터를 csv 파일로 저장

In [21]:
esg_total.to_csv("../../data/ESG_Rating.csv", index=False)

In [22]:
pd.read_csv("../../data/ESG_Rating.csv")

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,95570.0,B,D,B,B,2021
1,AK홀딩스,6840.0,B+,B,B+,B+,2021
2,AP시스템,265520.0,D,D,D,C,2021
3,BGF,27410.0,A,A,A+,A,2021
4,BGF리테일,282330.0,A,A,A+,A,2021
...,...,...,...,...,...,...,...
6654,성신양회,4980.0,B이하,B+,B이하,B이하,2011
6655,성보화학,3080.0,NaN,B,NaN,NaN,2011
6656,성문전자,14910.0,B이하,C,B이하,B이하,2011
6657,선창산업,NaN,B이하,B,B이하,B이하,2011
